In [1]:
import pandas as pd
import geopandas as gpd
from meteostat import Stations, Hourly,Daily
from datetime import datetime
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic
import h3
import folium
import osmnx as ox
from shapely.geometry import Polygon

In [2]:
df = pd.read_csv('Taxi_Trips__2013-2023__20240713.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6495570 entries, 0 to 6495569
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Taxi ID                     object 
 2   Trip Start Timestamp        object 
 3   Trip End Timestamp          object 
 4   Trip Seconds                float64
 5   Trip Miles                  float64
 6   Pickup Census Tract         float64
 7   Dropoff Census Tract        float64
 8   Pickup Community Area       float64
 9   Dropoff Community Area      float64
 10  Fare                        float64
 11  Tips                        float64
 12  Tolls                       float64
 13  Extras                      float64
 14  Trip Total                  float64
 15  Payment Type                object 
 16  Company                     object 
 17  Pickup Centroid Latitude    float64
 18  Pickup Centroid Longitude   float64
 19  Pickup Centroid Locat

In [4]:
#Convert columns to string
df[['Pickup Census Tract','Dropoff Census Tract', 
    'Pickup Community Area','Dropoff Community Area']] = df[['Pickup Census Tract','Dropoff Census Tract', 
                                                             'Pickup Community Area','Dropoff Community Area']].fillna(0).astype(int).astype(str).replace('0', None)

df['Trip Start Timestamp'] = pd.to_datetime(df['Trip Start Timestamp'])
df['Trip End Timestamp'] = pd.to_datetime(df['Trip End Timestamp'])

df.drop('Community Areas', axis=1, inplace=True)

In [5]:
df.isnull().mean() * 100

Trip ID                        0.000000
Taxi ID                        0.002386
Trip Start Timestamp           0.000000
Trip End Timestamp             0.001278
Trip Seconds                   0.019783
Trip Miles                     0.000308
Pickup Census Tract           55.679271
Dropoff Census Tract          56.811211
Pickup Community Area          3.726186
Dropoff Community Area         9.538316
Fare                           0.186943
Tips                           0.186943
Tolls                          0.186943
Extras                         0.186943
Trip Total                     0.186943
Payment Type                   0.000000
Company                        0.000000
Pickup Centroid Latitude       3.689407
Pickup Centroid Longitude      3.689407
Pickup Centroid Location       3.689407
Dropoff Centroid Latitude      8.928300
Dropoff Centroid Longitude     8.928300
Dropoff Centroid  Location     8.928300
dtype: float64

Definition: The longitude (latitude) of the center of the pickup census tract or the community area if the census tract has been hidden for privacy. This column often will be blank for locations outside Chicago.

--> Two issues:
- 1. Null values for locations outside Chicago
  2. Mix longitude (latitude) of census tract and community area

--> If "Pickup Census Tract" is null, the longitude (latitude) is from "Community Area". Otherwise, the longitude (latitude) is from "Census Tract"

Strategy: 
- It is required to use "Census Tract" + No way to impute missing "Census Tract" --> Drop all missing values in Census Tract
- Use information from "Census Tract" to fill in missing values in "Community Areas" and longitudes and latitudes.
- Can drop missing values in other columns for simplicity


# 1. Filling missing values

In [6]:
df = df.dropna(subset=['Pickup Census Tract', 'Dropoff Census Tract'])

In [7]:
df.isnull().mean() * 100

Trip ID                       0.000000
Taxi ID                       0.000000
Trip Start Timestamp          0.000000
Trip End Timestamp            0.000000
Trip Seconds                  0.015837
Trip Miles                    0.000253
Pickup Census Tract           0.000000
Dropoff Census Tract          0.000000
Pickup Community Area         0.112449
Dropoff Community Area        1.638250
Fare                          0.262237
Tips                          0.262237
Tolls                         0.262237
Extras                        0.262237
Trip Total                    0.262237
Payment Type                  0.000000
Company                       0.000000
Pickup Centroid Latitude      0.028572
Pickup Centroid Longitude     0.028572
Pickup Centroid Location      0.028572
Dropoff Centroid Latitude     0.254156
Dropoff Centroid Longitude    0.254156
Dropoff Centroid  Location    0.254156
dtype: float64

In [8]:
# Load the shapefiles or GeoJSON files
census_tracts = gpd.read_file('census_tract/geo_export_0caa4a0b-c5b9-4cef-b66d-72eced332409.shp')
#community_areas = gpd.read_file('community_area/geo_export_0ec1e45e-aa60-4dc6-a5c8-cb395ee19c94.shp')

census_tracts['count_areas'] = census_tracts.groupby('geoid10')['commarea'].transform('nunique')

census_tracts['count_areas'].unique()

array([1])

In Chicago, one Census Tract belongs exactly to one Community Area, so we can fill missing Community Area with Census Tract

In [9]:
census_tracts['centroid'] = census_tracts.geometry.centroid
census_tracts['latitude'] = census_tracts['centroid'].y
census_tracts['longitude'] = census_tracts['centroid'].x

In [10]:
hex_resolutions = [6, 7, 8] # <--- Add or remove resolution here 

In [11]:
def add_hexagon_data(gdf, hex_resolutions):
    gdf = gdf.copy()

    for res in hex_resolutions:
        hex_id_column = f'hex_id_{res}'
        hex_geometry_column = f'hex_geometry_{res}'

        gdf[hex_id_column] = gdf['centroid'].apply(lambda x: h3.geo_to_h3(x.y, x.x, res))

        gdf[hex_geometry_column] = gdf[hex_id_column].apply(
            lambda x: Polygon([(p[0], p[1]) for p in h3.h3_to_geo_boundary(x, geo_json=True)])
        )

        gdf = gpd.GeoDataFrame(gdf, geometry=hex_geometry_column)
        gdf[f'lon_{res}'] = gdf[hex_geometry_column].centroid.x
        gdf[f'lat_{res}'] = gdf[hex_geometry_column].centroid.y

    return gdf

census_tracts = add_hexagon_data(census_tracts, hex_resolutions)

In [12]:
hex_ids = []
for res in hex_resolutions:
    hex_ids.append(f'hex_id_{res}')
df = pd.merge(df, census_tracts[['geoid10','commarea','centroid','latitude','longitude'] + hex_ids], left_on='Pickup Census Tract', right_on='geoid10')

df['Pickup Community Area'] = df['Pickup Community Area'].fillna(df['commarea'])
df['Pickup Centroid Latitude'] = df['Pickup Centroid Latitude'].fillna(df['latitude'])
df['Pickup Centroid Longitude'] = df['Pickup Centroid Longitude'].fillna(df['longitude'])
df['Pickup Centroid Location'] = df['Pickup Centroid Location'].fillna(df['centroid'])
df = df.drop(columns=['geoid10','commarea','centroid','latitude','longitude'])

df = pd.merge(df, census_tracts[['geoid10','commarea','centroid','latitude','longitude']], left_on='Dropoff Census Tract', right_on='geoid10')
df['Dropoff Community Area'] = df['Dropoff Community Area'].fillna(df['commarea'])
df['Dropoff Centroid Latitude'] = df['Dropoff Centroid Latitude'].fillna(df['latitude'])
df['Dropoff Centroid Longitude'] = df['Dropoff Centroid Longitude'].fillna(df['longitude'])
df['Dropoff Centroid  Location'] = df['Dropoff Centroid  Location'].fillna(df['centroid'])
df = df.drop(columns=['geoid10','commarea','centroid','latitude','longitude'])

df = df.dropna()

In [13]:
df.isnull().mean() * 100

Trip ID                       0.0
Taxi ID                       0.0
Trip Start Timestamp          0.0
Trip End Timestamp            0.0
Trip Seconds                  0.0
Trip Miles                    0.0
Pickup Census Tract           0.0
Dropoff Census Tract          0.0
Pickup Community Area         0.0
Dropoff Community Area        0.0
Fare                          0.0
Tips                          0.0
Tolls                         0.0
Extras                        0.0
Trip Total                    0.0
Payment Type                  0.0
Company                       0.0
Pickup Centroid Latitude      0.0
Pickup Centroid Longitude     0.0
Pickup Centroid Location      0.0
Dropoff Centroid Latitude     0.0
Dropoff Centroid Longitude    0.0
Dropoff Centroid  Location    0.0
hex_id_6                      0.0
hex_id_7                      0.0
hex_id_8                      0.0
dtype: float64

In [14]:
df

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,hex_id_6,hex_id_7,hex_id_8
0,716d7a0a2a097facc3f0f63e326830ecdf923d0a,2d72c5e6313ad93f663008a55045cad0c76164b057dcb7...,2023-12-31 23:45:00,2024-01-01 00:00:00,649.0,2.57,17031833000,17031330100,28,33,...,City Service,41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),41.859350,-87.617358,POINT (-87.6173580061 41.859349715),862664cafffffff,872664cadffffff,882664cad7fffff
1,8142f4b1547f4e4a683a80b5a6c7d0325ce09559,f75191fdf728d7ed7f4277ee1e39372c16658b87abc26a...,2023-12-31 23:45:00,2024-01-01 00:00:00,600.0,1.10,17031320100,17031320400,32,32,...,Chicago Independents,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),862664c1fffffff,872664c1effffff,882664c1e3fffff
2,b68a7310d2ba573ce09f55fa546408264e0b3dd7,9454a3cb5d7e8ef84ca9bfff9f1a5d235021fef66b41a8...,2023-12-31 23:45:00,2024-01-01 00:00:00,755.0,2.37,17031833000,17031081201,28,8,...,City Service,41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),862664cafffffff,872664cadffffff,882664cad7fffff
3,ec183abaa7ff142f17ebcdafa1f3d4e611a9f494,f6d1b6c930d62f6d8cbbd8f86a593ff057408c82f76474...,2023-12-31 23:45:00,2024-01-01 00:00:00,786.0,2.02,17031081500,17031330100,8,33,...,Chicago Independents,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.859350,-87.617358,POINT (-87.6173580061 41.859349715),862664c1fffffff,872664c1effffff,882664c1e1fffff
4,ed445ada05f17c5f359892eda3c329e1445b5e7b,4b034948aceedd53262ae713f864b0364953a1852b6b24...,2023-12-31 23:45:00,2023-12-31 23:45:00,4.0,0.00,17031320100,17031320100,32,32,...,Sun Taxi,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),862664c1fffffff,872664c1effffff,882664c1e3fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726032,3f07cb261574b204709ba5337494527094a9bae4,4ab7a7510c1ebcc9b2e3eaa7bdd6508dbea34da7986aca...,2023-01-01 00:00:00,2023-01-01 00:15:00,1341.0,16.63,17031980000,17031081500,76,8,...,Sun Taxi,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),862759347ffffff,87275934effffff,88275934edfffff
2726033,61ddfa9c7fecac1b43962d8447fa930371377925,90738dbca5b9c7c2de984bf7e96a81569178364ca8cd29...,2023-01-01 00:00:00,2023-01-01 00:00:00,384.0,0.48,17031081500,17031081403,8,8,...,Flash Cab,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),862664c1fffffff,872664c1effffff,882664c1e1fffff
2726034,5fa9587952ed348823fc68ce1e25f5cd031a5961,0602c4dcde4b0fa95e24da18797128a90565512392fffd...,2023-01-01 00:00:00,2023-01-01 00:30:00,1764.0,15.46,17031980000,17031071500,76,7,...,Medallion Leasin,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.914616,-87.631717,POINT (-87.6317173661 41.9146162864),862759347ffffff,87275934effffff,88275934edfffff
2726035,6e26e8341d032293c24990114d321d1607a83fd2,dba6a86e74669ab2eb5130718c23bf1800b66a4ec88836...,2023-01-01 00:00:00,2023-01-01 00:15:00,734.0,0.87,17031839100,17031081700,32,8,...,5 Star Taxi,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),862664c1fffffff,872664c1affffff,882664c1a9fffff


# 2. Aggregation

## a. Census tracts

In [15]:
df['Date'] = df['Trip Start Timestamp'].dt.date
df['Hour'] = df['Trip Start Timestamp'].dt.strftime('%Y-%m-%d %H:00:00') 

aggregation_functions = {
     'Trip ID': 'count',
    'Trip Seconds': 'sum',
    'Trip Miles': 'sum',
    'Fare': 'sum',
    'Tips': 'sum',
    'Tolls': 'sum',
    'Extras': 'sum',
    'Trip Total': 'sum'
}

# Daily aggregation
daily_tract_agg = df.groupby(['Date', 'Pickup Census Tract']).agg(aggregation_functions)
daily_tract_agg = daily_tract_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()

# Hourly aggregation
hourly_tract_agg = df.groupby(['Hour', 'Pickup Census Tract']).agg(aggregation_functions)
hourly_tract_agg = hourly_tract_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()

## b. Hexagons

In [16]:
# Hexagon 6
daily_hex6_agg = df.groupby(['Date', 'hex_id_6']).agg(aggregation_functions)
daily_hex6_agg = daily_hex6_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()
hourly_hex6_agg = df.groupby(['Hour', 'hex_id_6']).agg(aggregation_functions)
hourly_hex6_agg = hourly_hex6_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()

# Hexagon 7
daily_hex7_agg = df.groupby(['Date', 'hex_id_7']).agg(aggregation_functions)
daily_hex7_agg = daily_hex7_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()
hourly_hex7_agg = df.groupby(['Hour', 'hex_id_7']).agg(aggregation_functions)
hourly_hex7_agg = hourly_hex7_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()

# Hexagon 8
daily_hex8_agg = df.groupby(['Date', 'hex_id_8']).agg(aggregation_functions)
daily_hex8_agg = daily_hex8_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()
hourly_hex8_agg = df.groupby(['Hour', 'hex_id_8']).agg(aggregation_functions)
hourly_hex8_agg = hourly_hex8_agg.rename(columns={'Trip ID': 'Number of Trips'}).reset_index()

# 3. POI data

In [17]:
def hexagon_to_polygon(hex_id):
    boundary = h3.h3_to_geo_boundary(hex_id, geo_json=True)
    polygon = Polygon([(coord[0], coord[1]) for coord in boundary])  
    return gpd.GeoDataFrame({'geometry': [polygon]}, index=[0])

In [18]:
places = ['restaurant','bar', 'biergarten', 'fast_food', 'ice_cream', 'pub', 'cafe',
          'university','college', 'kindergarten', 'library', 'school', 
          'taxi', 'bank', 'hospital','casino','cinema','nightclub'] ##<-- Add or remove POI data here

## a. Census tracts

In [19]:
bbox = census_tracts.unary_union.bounds
north, south, east, west = bbox[3], bbox[1], bbox[2], bbox[0]

pois = ox.features_from_bbox(north, south, east, west, tags={'amenity': True})
relevant_pois = pois[pois['amenity'].isin(places)]

census_tracts.crs = "EPSG:4326"
relevant_pois = relevant_pois.to_crs(census_tracts.crs) #ensure CRS match

pois_with_tracts = gpd.sjoin(relevant_pois, census_tracts, how='left', predicate='within')
poi_counts = pois_with_tracts.groupby(['geoid10', 'amenity']).size().unstack(fill_value=0).reset_index()

daily_tract_agg = pd.merge(daily_tract_agg, poi_counts, left_on='Pickup Census Tract', right_on='geoid10', how='left')
daily_tract_agg[places] = daily_tract_agg[places].fillna(0)
daily_tract_agg.drop('geoid10',axis=1,inplace=True)

hourly_tract_agg = pd.merge(hourly_tract_agg, poi_counts, left_on='Pickup Census Tract', right_on='geoid10', how='left')
hourly_tract_agg[places] = hourly_tract_agg[places].fillna(0)
hourly_tract_agg.drop('geoid10',axis=1,inplace=True)

## b. Hexagons

In [20]:
# Hexagon 6
poi_counts_hex6 = pois_with_tracts.groupby(['hex_id_6', 'amenity']).size().unstack(fill_value=0).reset_index()
daily_hex6_agg = pd.merge(daily_hex6_agg, poi_counts_hex6, on='hex_id_6', how='left')
daily_hex6_agg[places] = daily_hex6_agg[places].fillna(0)
hourly_hex6_agg = pd.merge(hourly_hex6_agg, poi_counts_hex6, on='hex_id_6', how='left')
hourly_hex6_agg[places] = hourly_hex6_agg[places].fillna(0)

# Hexagon 7
poi_counts_hex7 = pois_with_tracts.groupby(['hex_id_7', 'amenity']).size().unstack(fill_value=0).reset_index()
daily_hex7_agg = pd.merge(daily_hex7_agg, poi_counts_hex7, on='hex_id_7', how='left')
daily_hex7_agg[places] = daily_hex7_agg[places].fillna(0)
hourly_hex7_agg = pd.merge(hourly_hex7_agg, poi_counts_hex7, on='hex_id_7', how='left')
hourly_hex7_agg[places] = hourly_hex7_agg[places].fillna(0)

# Hexagon 8
poi_counts_hex8 = pois_with_tracts.groupby(['hex_id_8', 'amenity']).size().unstack(fill_value=0).reset_index()
daily_hex8_agg = pd.merge(daily_hex8_agg, poi_counts_hex8, on='hex_id_8', how='left')
daily_hex8_agg[places] = daily_hex8_agg[places].fillna(0)
hourly_hex8_agg = pd.merge(hourly_hex8_agg, poi_counts_hex8, on='hex_id_8', how='left')
hourly_hex8_agg[places] = hourly_hex8_agg[places].fillna(0)

# 4. Weather data

In [21]:
start = datetime(2023, 1, 1)
end = datetime(2024, 1, 1)

# Get weather stations in Chicago
stations = Stations()
center_lat = census_tracts.geometry.centroid.y.mean()
center_lon = census_tracts.geometry.centroid.x.mean()
chicago_stations = stations.nearby(center_lat, center_lon)  
station_list = chicago_stations.fetch(6) #6 closest stations in Chicago
station_list = station_list.drop('KCGX0', axis=0) #Remove one unavailable station

hourly_weather_data = Hourly(list(station_list.index), start, end) 
hourly_weather_data = hourly_weather_data.fetch()
hourly_weather_data.reset_index(inplace =True)
hourly_weather_data['time'] = pd.to_datetime(hourly_weather_data['time'])
hourly_weather_data.drop(['snow','wpgt','tsun'],axis=1,inplace=True) #weather stations do not track this info

daily_weather_data = Daily(list(station_list.index), start, end) 
daily_weather_data = daily_weather_data.fetch()
daily_weather_data.reset_index(inplace =True)
daily_weather_data['time'] = pd.to_datetime(daily_weather_data['time'])
daily_weather_data.drop(['snow','wpgt','tsun'],axis=1,inplace=True) #weather stations do not track this info

## a. Census tracts

In [22]:
# Idea from ChatGPT to shorten the time
gdf_stations = gpd.GeoDataFrame(station_list, geometry=gpd.points_from_xy(station_list.longitude, station_list.latitude))
gdf_stations.crs = "EPSG:4326"  # already used in census_tracts

gdf_census = gpd.GeoDataFrame(census_tracts, geometry=gpd.points_from_xy(census_tracts['longitude'], census_tracts['latitude']))
gdf_census.crs = "EPSG:4326"  

tree = cKDTree(np.array(list(zip(gdf_stations.geometry.x, gdf_stations.geometry.y))))

closest_stations = []
for point in gdf_census.geometry:
    dist, idx = tree.query(np.array([point.x, point.y]), k=1)
    closest_stations.append(gdf_stations.iloc[idx].name)

census_tracts['closest_station_census'] = closest_stations

In [23]:
daily_tract_agg = pd.merge(daily_tract_agg, census_tracts[['geoid10','closest_station_census']], left_on='Pickup Census Tract', right_on='geoid10', how='left')
daily_tract_agg['Date'] = pd.to_datetime(daily_tract_agg['Date'])
# Merge based on time and closest station
daily_tract_agg = pd.merge_asof(daily_tract_agg.sort_values('Date'), daily_weather_data.sort_values('time'), left_on='Date', right_on='time', left_by='closest_station_census',right_by='station', direction='nearest')

hourly_tract_agg = pd.merge(hourly_tract_agg, census_tracts[['geoid10','closest_station_census']], left_on='Pickup Census Tract', right_on='geoid10', how='left')
hourly_tract_agg['Hour'] = pd.to_datetime(hourly_tract_agg['Hour'])
hourly_tract_agg = pd.merge_asof(hourly_tract_agg.sort_values('Hour'), hourly_weather_data.sort_values('time'), left_on='Hour', right_on='time', left_by='closest_station_census',right_by='station', direction='nearest')

## b. Hexagons

In [24]:
# Hexagon 6
gdf_hex6 = gpd.GeoDataFrame(census_tracts, geometry=gpd.points_from_xy(census_tracts['lon_6'], census_tracts['lat_6']))
gdf_hex6.crs = "EPSG:4326" 
closest_stations = []
for point in gdf_hex6.geometry:
    dist, idx = tree.query(np.array([point.x, point.y]), k=1)
    closest_stations.append(gdf_stations.iloc[idx].name)
census_tracts['closest_station_hex6'] = closest_stations

# Hexagon 7
gdf_hex7 = gpd.GeoDataFrame(census_tracts, geometry=gpd.points_from_xy(census_tracts['lon_7'], census_tracts['lat_7']))
gdf_hex7.crs = "EPSG:4326" 
closest_stations = []
for point in gdf_hex7.geometry:
    dist, idx = tree.query(np.array([point.x, point.y]), k=1)
    closest_stations.append(gdf_stations.iloc[idx].name)
census_tracts['closest_station_hex7'] = closest_stations

# Hexagon 8
gdf_hex8 = gpd.GeoDataFrame(census_tracts, geometry=gpd.points_from_xy(census_tracts['lon_8'], census_tracts['lat_8']))
gdf_hex8.crs = "EPSG:4326" 
closest_stations = []
for point in gdf_hex8.geometry:
    dist, idx = tree.query(np.array([point.x, point.y]), k=1)
    closest_stations.append(gdf_stations.iloc[idx].name)
census_tracts['closest_station_hex8'] = closest_stations

In [25]:
# Hexagon 6
daily_hex6_agg = pd.merge(daily_hex6_agg, census_tracts[['hex_id_6','closest_station_hex6']].drop_duplicates(), on='hex_id_6', how='left')
daily_hex6_agg['Date'] = pd.to_datetime(daily_hex6_agg['Date'])
daily_hex6_agg = pd.merge_asof(daily_hex6_agg.sort_values('Date'), daily_weather_data.sort_values('time'), left_on='Date', right_on='time', left_by='closest_station_hex6',right_by='station', direction='nearest')

hourly_hex6_agg = pd.merge(hourly_hex6_agg, census_tracts[['hex_id_6','closest_station_hex6']].drop_duplicates(), on='hex_id_6', how='left')
hourly_hex6_agg['Hour'] = pd.to_datetime(hourly_hex6_agg['Hour'])
hourly_hex6_agg = pd.merge_asof(hourly_hex6_agg.sort_values('Hour'), hourly_weather_data.sort_values('time'), left_on='Hour', right_on='time', left_by='closest_station_hex6',right_by='station', direction='nearest')


# Hexagon 7
daily_hex7_agg = pd.merge(daily_hex7_agg, census_tracts[['hex_id_7','closest_station_hex7']].drop_duplicates(), on='hex_id_7', how='left')
daily_hex7_agg['Date'] = pd.to_datetime(daily_hex7_agg['Date'])
daily_hex7_agg = pd.merge_asof(daily_hex7_agg.sort_values('Date'), daily_weather_data.sort_values('time'), left_on='Date', right_on='time', left_by='closest_station_hex7',right_by='station', direction='nearest')

hourly_hex7_agg = pd.merge(hourly_hex7_agg, census_tracts[['hex_id_7','closest_station_hex7']].drop_duplicates(), on='hex_id_7', how='left')
hourly_hex7_agg['Hour'] = pd.to_datetime(hourly_hex7_agg['Hour'])
hourly_hex7_agg = pd.merge_asof(hourly_hex7_agg.sort_values('Hour'), hourly_weather_data.sort_values('time'), left_on='Hour', right_on='time', left_by='closest_station_hex7',right_by='station', direction='nearest')


# Hexagon 8
daily_hex8_agg = pd.merge(daily_hex8_agg, census_tracts[['hex_id_8','closest_station_hex8']].drop_duplicates(), on='hex_id_8', how='left')
daily_hex8_agg['Date'] = pd.to_datetime(daily_hex8_agg['Date'])
daily_hex8_agg = pd.merge_asof(daily_hex8_agg.sort_values('Date'), daily_weather_data.sort_values('time'), left_on='Date', right_on='time', left_by='closest_station_hex8',right_by='station', direction='nearest')

hourly_hex8_agg = pd.merge(hourly_hex8_agg, census_tracts[['hex_id_8','closest_station_hex8']].drop_duplicates(), on='hex_id_8', how='left')
hourly_hex8_agg['Hour'] = pd.to_datetime(hourly_hex8_agg['Hour'])
hourly_hex8_agg = pd.merge_asof(hourly_hex8_agg.sort_values('Hour'), hourly_weather_data.sort_values('time'), left_on='Hour', right_on='time', left_by='closest_station_hex8',right_by='station', direction='nearest')

# 5. Export csv files

In [26]:
# Drop unnecessary (duplicated) columns
daily_tract_agg.drop(['closest_station_census', 'station', 'geoid10','time'], axis=1, inplace=True)
hourly_tract_agg.drop(['closest_station_census', 'station', 'geoid10','time'], axis=1, inplace=True)
daily_hex6_agg.drop(['closest_station_hex6', 'station','time'], axis=1, inplace=True)
hourly_hex6_agg.drop(['closest_station_hex6', 'station','time'], axis=1, inplace=True)
daily_hex7_agg.drop(['closest_station_hex7', 'station','time'], axis=1, inplace=True)
hourly_hex7_agg.drop(['closest_station_hex7', 'station','time'], axis=1, inplace=True)
daily_hex8_agg.drop(['closest_station_hex8', 'station','time'], axis=1, inplace=True)
hourly_hex8_agg.drop(['closest_station_hex8', 'station','time'], axis=1, inplace=True)


daily_tract_agg.to_csv('census_tract_daily_data.csv', index=False)
hourly_tract_agg.to_csv('census_tract_hourly_data.csv', index=False)
daily_hex6_agg.to_csv('hexagon6_daily_data.csv', index=False)
hourly_hex6_agg.to_csv('hexagon6_hourly_data.csv', index=False)
daily_hex7_agg.to_csv('hexagon7_daily_data.csv', index=False)
hourly_hex7_agg.to_csv('hexagon7_hourly_data.csv', index=False)
daily_hex8_agg.to_csv('hexagon8_daily_data.csv', index=False)
hourly_hex8_agg.to_csv('hexagon8_hourly_data.csv', index=False)

In [ ]:
#Map

In [19]:
def plot_hexagons(hex_ids, map_object):
    for hex_id in hex_ids:
        hex_boundary = h3.h3_to_geo_boundary(hex_id, geo_json=True)
        hex_boundary = [(coord[1], coord[0]) for coord in hex_boundary]
        folium.Polygon(locations=hex_boundary, color='blue', fill=True, fill_opacity=0.5).add_to(map_object)

center_lat = census_tracts.geometry.centroid.y.mean()
center_lon = census_tracts.geometry.centroid.x.mean()
map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

for hex_list in census_tracts['hexagons_9']:
    plot_hexagons(hex_list, map)

map